In [ ]:
#@title **RUN THIS CELL** (once for runtime)
#@markdown it will take 2 min to run the cell
import os
os.system("pip3 install pillow")
os.system("sudo apt-get install -y poppler-utils")
os.system("pip3 install pytesseract")
os.system("pip3 install pdf2image")
os.system("sudo apt-get install tesseract-ocr")
os.system("sudo apt install libreoffice")

In [ ]:
#@title **Convert Pdf to Doc/txt**

# Import libraries
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
import shutil
from google.colab import files

#Delete any temp directory that priviously exist
try:
  shutil.rmtree("/content/temp")
except:
  pass


#Create Temp directory
os.makedirs("/content/temp")
#make temp directory as current working directory
os.chdir("/content/temp/")

#upload file from local machine
print("Please Upload Pdf File:")
PDF_file = files.upload()

#get file name from uploaded file
PDF_file=list(PDF_file.keys())[0]

#select output type

#@markdown **1. Select the output file type doc or txt**
Convert_To_Type = "Doc" #@param ["Doc", "Txt"]
#@markdown **Note: For Pdf with image, doc will also be of image type** (choose txt for editable txt *may contain junk)

# if output file is doc
if (Convert_To_Type=="Doc"):
  
  #convert pdf file to doc file using libre office
  os.system(f"libreoffice --infilter='writer_pdf_import' --convert-to doc {PDF_file}")
  
  #download resulting doc file
  files.download(PDF_file[0:-3]+"doc")

#if output file is txt  
else:

  #convert pages of pdf to image
  pages = convert_from_path(PDF_file, 500)
  #variable to count number of image
  image_counter = 1

  #Saving images with successive names
  for page in pages:
    filename = "page_"+str(image_counter)+".jpg"
    page.save(filename, 'JPEG')
    image_counter = image_counter + 1

  #get total number of images
  filelimit = image_counter-1

  #Name for output file
  outfile = PDF_file[0:-3]+"txt"

  #opening output file as apendable file
  f = open(outfile, "a")

  #iterate along the images and get the text using tessaract package
  for i in range(1, filelimit + 1):
    filename = "page_"+str(i)+".jpg" 
    #get text string from  each image(page)
    text = str(((pytesseract.image_to_string(Image.open(filename)))))
    #removing next line junk that comes after conversion
    text = text.replace('-\n', '')    
    #append string to opened file
    f.write(text)

  #after iterating through all pages close the file  
  f.close()

  # download resulting text file
  files.download(PDF_file[0:-3]+"txt")
  